# Determine KLM Operators
### Author: Sarah, Claudia
### Reviewer: Claudia

## Structured Analysis
### Import all relevant libraries

In [1]:
!pip3 install pingouin

In [20]:
import pandas as pd
import pingouin as pg
import seaborn as sns
from matplotlib import pyplot as plt

In [21]:
from PyQt5.QtCore import QDateTime
from PyQt5.QtCore import Qt

## Function for calculating time difference

In [22]:
def calc_time_diff(data):
    start_time = QDateTime.fromString(data["timestamp"][1], Qt.ISODateWithMs)
    start_time_in_mm = QDateTime.toMSecsSinceEpoch(start_time)
    
    number = len(data) - 1
    end_time = QDateTime.fromString(data["timestamp"][number], Qt.ISODateWithMs)
    end_time_in_mm = QDateTime.toMSecsSinceEpoch(end_time)
    
    return end_time_in_mm - start_time_in_mm

## K-Operator
In order to calculate the time for a keystroke, one key (1) is pressed 100 times in succession.
The timestamp is logged when a key is pressed.

### Read csv file

In [23]:
raw_data_keystroke = pd.read_csv("klm_csv/keystroke.csv")

In [24]:
raw_data_keystroke

,input_type,input_value,timestamp
0,key_pressed,1,2021-05-21 19:35:57.603895
1,key_pressed,1,2021-05-21 19:35:57.860989
2,key_pressed,1,2021-05-21 19:35:58.157107
3,key_pressed,1,2021-05-21 19:35:58.381104
4,key_pressed,1,2021-05-21 19:35:58.717120
...,...,...,...
95,key_pressed,1,2021-05-21 19:36:24.209077
96,key_pressed,1,2021-05-21 19:36:24.440689
97,key_pressed,1,2021-05-21 19:36:24.705082
98,key_pressed,1,2021-05-21 19:36:24.928822


### Calculate time for one K
For calculation of the first timestamp is ignored, because you need time for orientation.
The time difference between the last and the second timestamp is calculated.
The result is divided by the number of keystrokes (equal to the length of the dataframe) minus one to get the time for one keystroke. 

In [68]:
time_diff_k = calc_time_diff(raw_data_keystroke)
time_diff_k

27308

In [69]:
time_for_one_keystroke = time_diff_k / (len(raw_data_keystroke) - 1)
time_for_one_keystroke

275.83838383838383

## B-Operator
To calculate the button click, the same button (2) is pressed 100 times in a row. 
In this case, it was logged when a button is clicked (press + release).

### Read csv file

In [70]:
raw_data_button_clicks = pd.read_csv("klm_csv/button_clicks.csv")

### Calculate time for one B
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated.
The result is divided by the number of button press and release (equal to the length of the dataframe) minus one.
To get the time for one button press or release the result is divided by two. 

In [71]:
time_diff_b = calc_time_diff(raw_data_button_clicks)
time_diff_b

31848

In [72]:
time_for_one_press = (time_diff_b / (len(raw_data_button_clicks) - 1)) / 2
time_for_one_press

160.84848484848484

## P-Operator
To calculate the pointing of the mouse, the mouse pointer is moved to each button at least once in arbitrary order. In total 100 button clicks are registered.
Each time a button is clicked the timestamp is logged.

### Read csv file

In [73]:
raw_data_pointing_mouse = pd.read_csv("klm_csv/pointing_mouse.csv")

### Calculate time for one P
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated and divided by the number of click entries minus one to get the time for one P including button click. To calculate one P the button click (press and release) is subtracted.

In [74]:
time_diff_p = calc_time_diff(raw_data_pointing_mouse)
time_diff_p

104087

In [75]:
time_for_one_pointing_with_b = time_diff_p / (len(raw_data_pointing_mouse) - 1)
time_for_one_pointing_with_b

1051.3838383838383

In [76]:
time_for_one_pointing = time_for_one_pointing_with_b - time_for_one_press * 2
time_for_one_pointing

729.6868686868686

## H-Operator
To calculate the time for switching between keyboard and mouse, it is alternated between pressing the key 1 and clicking the button 0. This was repeated 100 times so that each key/button was pressed/clicked 50 times. The position of the mouse was on the 0 button so that it was not necessary to move the mouse. 

### Read csv file

In [77]:
raw_data_hand_switch = pd.read_csv("klm_csv/hand_switch.csv")

### Calculate time for one H
The first timestamp is ignored as before.
The time difference between the last and the second timestamp is calculated.
To get the time for one handswitch, the average duration of the handswitches is calculated and then the time for one click (press and release) is subtracted.

In [78]:
time_diff_h = calc_time_diff(raw_data_hand_switch)
time_diff_h

62232

In [79]:
time_for_one_h_with_b = time_diff_h / (len(raw_data_hand_switch) - 1)
time_for_one_h_with_b

628.6060606060606

In [80]:
time_for_one_h = time_for_one_h_with_b - time_for_one_press * 2
time_for_one_h

306.90909090909093

## Overview of calculated KLM values
|Operator|time (s)|
|:-------|--------|
|Keystroke|0.28| 
|Pointing with mouse|0.73|
|Button press or release on mouse|0.16|
|Hand switching between keyboard and mouse |0.31|